## Утилита для парсинга конфигов в переменные окружения и обратно

Для настройки моделей используются конфиги большого уровня вложенности, параметры из которых периодически требуется устанавливать в качестве переменных окружения



In [ ]:
# Пример конфига
...

F0_path: "Utils/JDC/bst.t7"
ASR_config: "Utils/ASR/config.yml"
ASR_path: "Utils/ASR/epoch_00100.pth"

preprocess_params:
  sr: 24000
  spect_params:
    n_fft: 2048
    win_length: 1200
    hop_length: 300

model_params:
  dim_in: 64
  style_dim: 64
  latent_dim: 16
  num_domains: 20
  max_conv_dim: 512
  n_repeat: 4

...

In [ ]:
# Пример переменных окружения
...

F0_path=Utils/JDC/bst.t7
ASR_config=Utils/ASR/config.yml

...

preprocess_params.sr=24000
preprocess_params.spect_params.n_fft=2048

...

model_params.dim_in=64

...

**Псевдокод для парсинга конфига**
```
Преобразуем текст конфига в словарь (дерево)
Рекурсивно обходим дерево:
    Если значение это словарь:
        Рекурсивно обходим словарь
    Иначе:
        Формируем строку переменной окружения вида 
        "preprocess_params.sr.n_fft=2048" и добавляем к результату
Возвращает результат в виде строки
```

In [121]:
import yaml

def dic_to_env(yml_dict, env, str_key):
    env=''
    for key, value in yml_dict.items(): 
        if isinstance(value, dict):
            str_key = str_key + key + '.'
            env = env + dic_to_env(value, env, str_key)
            str_key = str_key.replace(key+'.', '') 
        else:
            env= env + str_key + key + '=' + str(value) + '\n'        
    return env  
            
def yaml_to_env(config_file: str) -> str:
    with open(config_file, 'r') as stream:
        yaml_dict = yaml.safe_load(stream)
    env=''
    env = dic_to_env(yml_dict, env, '')
    return env  

def env_to_yaml(env_list: str) -> str:
    env_str_list = env_list.split('\n')
    #yaml.safe_dump(d, sort_keys=False)
    return 

In [223]:
env_list = yaml_to_env('config.yml')
env_list

'log_dir=Models/VCTK20\nsave_freq=2\ndevice=cuda\nepochs=150\nbatch_size=5\npretrained_model=\nload_only_params=False\nfp16_run=True\ntrain_data=Data/train_list.txt\nval_data=Data/val_list.txt\nF0_path=Utils/JDC/bst.t7\nASR_config=Utils/ASR/config.yml\nASR_path=Utils/ASR/epoch_00100.pth\npreprocess_params.sr=24000\npreprocess_params.spect_params.n_fft=2048\npreprocess_params.spect_params.win_length=1200\npreprocess_params.spect_params.hop_length=300\nmodel_params.dim_in=64\nmodel_params.style_dim=64\nmodel_params.latent_dim=16\nmodel_params.num_domains=20\nmodel_params.max_conv_dim=512\nmodel_params.n_repeat=4\nmodel_params.w_hpf=0\nmodel_params.F0_channel=256\nloss_params.g_loss.lambda_sty=1.0\nloss_params.g_loss.lambda_cyc=5.0\nloss_params.g_loss.lambda_ds=1.0\nloss_params.g_loss.lambda_norm=1.0\nloss_params.g_loss.lambda_asr=10.0\nloss_params.g_loss.lambda_f0=5.0\nloss_params.g_loss.lambda_f0_sty=0.1\nloss_params.g_loss.lambda_adv=2.0\nloss_params.g_loss.lambda_adv_cls=0.5\nloss_par

In [ ]:
env_to_yaml(env_list)

In [126]:
with open('config.yml', 'r') as stream:
    yml_dict = yaml.safe_load(stream)
yml_dict

{'log_dir': 'Models/VCTK20',
 'save_freq': 2,
 'device': 'cuda',
 'epochs': 150,
 'batch_size': 5,
 'pretrained_model': '',
 'load_only_params': False,
 'fp16_run': True,
 'train_data': 'Data/train_list.txt',
 'val_data': 'Data/val_list.txt',
 'F0_path': 'Utils/JDC/bst.t7',
 'ASR_config': 'Utils/ASR/config.yml',
 'ASR_path': 'Utils/ASR/epoch_00100.pth',
 'preprocess_params': {'sr': 24000,
  'spect_params': {'n_fft': 2048, 'win_length': 1200, 'hop_length': 300}},
 'model_params': {'dim_in': 64,
  'style_dim': 64,
  'latent_dim': 16,
  'num_domains': 20,
  'max_conv_dim': 512,
  'n_repeat': 4,
  'w_hpf': 0,
  'F0_channel': 256},
 'loss_params': {'g_loss': {'lambda_sty': 1.0,
   'lambda_cyc': 5.0,
   'lambda_ds': 1.0,
   'lambda_norm': 1.0,
   'lambda_asr': 10.0,
   'lambda_f0': 5.0,
   'lambda_f0_sty': 0.1,
   'lambda_adv': 2.0,
   'lambda_adv_cls': 0.5,
   'norm_bias': 0.5},
  'd_loss': {'lambda_reg': 1.0, 'lambda_adv_cls': 0.1, 'lambda_con_reg': 10.0},
  'adv_cls_epoch': 50,
  'con_reg

In [284]:
def represents_int(s):
    try: 
        int(s)
    except ValueError:
        return False
    else:
        return True
    

def from_str_to_type(value: str):
    if represents_int(value):
        return int(value)
    if re.findall(r'[\d]*[.][\d]+', value):
        return float(value)
    if (s == "True"):
        return True
    elif (s == "False"):
        return false
    return value

In [285]:
from_str_to_type('Models/VCTK20')

'Models/VCTK20'

In [286]:
def _env_to_yaml(env_str: str, config: dict): 
    env_dict = {}
    key, value = env_str.split('=')
    value = from_str_to_type(value)
    if ('.' in key):
        key_0 = key.split('.')[0]
        new_str = '.'.join(env_str.split('.')[1:])
        if (key_0 in config):
            config[key_0].update(_env_to_yaml(new_str, config[key_0]))
        else:    
            env_dict[key_0] = _env_to_yaml(new_str, config)

    else:
        env_dict[key]= value
    return env_dict    

In [287]:
import re

def env_to_yaml(env_list: str):
    env_dict = {}
    env_str_list = env_list.split('\n')[:-1]
    for env_str in env_str_list:
        d = _env_to_yaml(env_str, env_dict)
        env_dict.update(d)  
    yaml.dump(env_dict)     
    return env_dict

In [288]:
env_to_yaml(env_list)

{'log_dir': 'Models/VCTK20',
 'save_freq': 2,
 'device': 'cuda',
 'epochs': 150,
 'batch_size': 5,
 'pretrained_model': '',
 'load_only_params': True,
 'fp16_run': True,
 'train_data': 'Data/train_list.txt',
 'val_data': 'Data/val_list.txt',
 'F0_path': 'Utils/JDC/bst.t7',
 'ASR_config': 'Utils/ASR/config.yml',
 'ASR_path': 'Utils/ASR/epoch_00100.pth',
 'preprocess_params': {'sr': 24000,
  'spect_params': {'n_fft': 2048, 'win_length': 1200, 'hop_length': 300}},
 'model_params': {'dim_in': 64,
  'style_dim': 64,
  'latent_dim': 16,
  'num_domains': 20,
  'max_conv_dim': 512,
  'n_repeat': 4,
  'w_hpf': 0,
  'F0_channel': 256},
 'loss_params': {'g_loss': {'lambda_sty': 1.0,
   'lambda_cyc': 5.0,
   'lambda_ds': 1.0,
   'lambda_norm': 1.0,
   'lambda_asr': 10.0,
   'lambda_f0': 5.0,
   'lambda_f0_sty': 0.1,
   'lambda_adv': 2.0,
   'lambda_adv_cls': 0.5,
   'norm_bias': 0.5}},
 'd_loss': {'lambda_reg': 1.0, 'lambda_adv_cls': 0.1, 'lambda_con_reg': 10.0},
 'adv_cls_epoch': 50,
 'con_reg_ep

In [ ]:

def env_to_yaml(env_list: str, config: dict={}):
    env_dict = {}
    
    env_str_list = env_list.split('\n')[:-1]
    for env_str in env_str_list:
        key, value = env_str.split('=')
        if ('.' in key):
            key_0 = key.split('.')[0]
            new_str = str(env_str.split('.')[1:])
            
            config[key_0] = new_str
            env_dict[key_0] = env_to_yaml(new_str, config)
           
        else:
            env_dict[key]= value  
    return env_dict 

In [142]:
yaml.safe_dump(env_dict_list, sort_keys=False)

"- log_dir: Models/VCTK20\n- save_freq: '2'\n- device: cuda\n- epochs: '150'\n- batch_size: '5'\n- pretrained_model: ''\n- load_only_params: 'False'\n- fp16_run: 'True'\n- train_data: Data/train_list.txt\n- val_data: Data/val_list.txt\n- F0_path: Utils/JDC/bst.t7\n- ASR_config: Utils/ASR/config.yml\n- ASR_path: Utils/ASR/epoch_00100.pth\n- preprocess_params:\n    sr: '24000'\n- preprocess_params:\n    spect_params: '2048'\n- preprocess_params:\n    spect_params: '1200'\n- preprocess_params:\n    spect_params: '300'\n- model_params:\n    dim_in: '64'\n- model_params:\n    style_dim: '64'\n- model_params:\n    latent_dim: '16'\n- model_params:\n    num_domains: '20'\n- model_params:\n    max_conv_dim: '512'\n- model_params:\n    n_repeat: '4'\n- model_params:\n    w_hpf: '0'\n- model_params:\n    F0_channel: '256'\n- loss_params:\n    g_loss: '1.0'\n- loss_params:\n    g_loss: '5.0'\n- loss_params:\n    g_loss: '1.0'\n- loss_params:\n    g_loss: '1.0'\n- loss_params:\n    g_loss: '10.0'\